In [74]:
import os
import openai
import logging
import sys

from dotenv import load_dotenv
from sqlalchemy import create_engine
from llama_index.core import download_loader,VectorStoreIndex,PromptTemplate
from llama_index.readers.database import DatabaseReader
from llama_index.readers.file import PyMuPDFReader
from pathlib import Path
from IPython.display import Markdown, display
from langchain import hub

load_dotenv()  

True

In [75]:
connection_string = f"{os.getenv('DB_SCHEME')}://{os.getenv('DB_USER')}:{os.getenv('DB_PASS')}@{os.getenv('DB_HOST')}/{os.getenv('DB_NAME')}?driver=ODBC+Driver+17+for+SQL+Server"
engine = create_engine(connection_string)
db = DatabaseReader(engine)

In [78]:
query = """
SELECT TOP (3) 
       [LastName]
      ,[FirstName]
      ,[Title]
  FROM [master].[dbo].[Employees]
"""
documents = db.load_data(query=query)
print(documents)

[Document(id_='5d28b018-fcf6-475d-8311-476043d6aedc', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='LastName: Davolio, FirstName: Nancy, Title: Sales Representative', mimetype=None, path=None, url=None), image_resource=None, audio_resource=None, video_resource=None, text_template='{metadata_str}\n\n{content}'), Document(id_='8fc63fc4-fdaa-49b3-bc6d-635c79e6342e', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='LastName: Fuller, FirstName: Andrew, Title: Vice President, Sales', mimetype=None, path=None, url=None), image_resource=None, audio_resource=None, video_resource=None, text_template='{metadata_str}\n\n{content}')

In [44]:
openai.api_key =  os.getenv('OPENAI_API_KEY')
logging.basicConfig(stream = sys.stdout , level = logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream = sys.stdout))

In [ ]:
# loader = PyMuPDFReader()
# documents  = loader.load(r'data\test_doc.txt')
# print(documents)

[Document(id_='bf0de311-544e-43cf-b8f6-6922e745c0b4', embedding=None, metadata={'total_pages': 1, 'file_path': 'data\\test_doc.txt', 'source': '1'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='Run-time steering strategies like Medprompt are\nvaluable for guiding large language models\n(LLMs) to top performance on challenging tasks.\nMedprompt demonstrates that a general LLM\ncan be focused to deliver state-of-the-art\nperformance on specialized domains like medicine by\nusing a prompt to elicit a run-time strategy\ninvolving chain of thought reasoning and ensembling.\nOpenAI’s o1-preview model represents a new paradigm,\nwhere a model is designed to do run-time\nreasoning before generating final responses. We seek\nto understand the behavior of o1-preview\non a diverse set of medical challenge problem\nbenchmarks. Following on 

In [79]:
documents

[Document(id_='5d28b018-fcf6-475d-8311-476043d6aedc', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='LastName: Davolio, FirstName: Nancy, Title: Sales Representative', mimetype=None, path=None, url=None), image_resource=None, audio_resource=None, video_resource=None, text_template='{metadata_str}\n\n{content}'),
 Document(id_='8fc63fc4-fdaa-49b3-bc6d-635c79e6342e', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='LastName: Fuller, FirstName: Andrew, Title: Vice President, Sales', mimetype=None, path=None, url=None), image_resource=None, audio_resource=None, video_resource=None, text_template='{metadata_str}\n\n{content}'

In [80]:
from llama_index.llms.openai import OpenAI

gpt35_llm  = OpenAI(model="gpt-3.5-turbo")
index = VectorStoreIndex.from_documents(documents)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [81]:
query_str = "Tell me Nancys title?"
query_engine  = index.as_query_engine(similarity_top_k=2, llm=gpt35_llm)
vector_retriever  = index.as_retriever(similarity_top_k = 2)

In [82]:
response = query_engine.query(query_str)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [84]:
response

Response(response='Sales Representative', source_nodes=[NodeWithScore(node=TextNode(id_='36dfbc98-8dc1-4b61-8b6e-c719067e41bc', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='5d28b018-fcf6-475d-8311-476043d6aedc', node_type='4', metadata={}, hash='7fa8d733c8dcd7be4aef10b635cea6fa0f609ea92ced7a0cde9d8ffc0d45acb4')}, metadata_template='{key}: {value}', metadata_separator='\n', text='LastName: Davolio, FirstName: Nancy, Title: Sales Representative', mimetype='text/plain', start_char_idx=0, end_char_idx=64, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'), score=0.8287994765996185), NodeWithScore(node=TextNode(id_='002fd2ec-00a1-46db-843d-5be27755ede9', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='729f0969-c4aa-4286-b8f1-e5052ac240e0',